In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils, models
from torchsummary import summary
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from PIL import Image

/tmp/ipykernel_227/1874384687.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
model = models.resnet18(pretrained=True).to(device)

# Freezing the base model layers to prevent retraining
for param in model.parameters():
    param.requires_grad = False

/home/cardon/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cardon/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Notes on the Resnet Model:

** convert images to b&w
- Requires input images dimensions (256,256) ** resize our images
- My additional fully connected layer needs dimensions (2048,10) - 10 for the 10 classes for the 10 style types (--)
- Image preprocessing requires:
  1. (224,224) center crop
  2. image is normalized with mean = 255*[0.485, 0.456, 0.406] and
  std = 255*[0.229, 0.224, 0.225]
  3. transpose it from HWC to CHW layout
- Post-processing involves calculating the softmax probability scores for each class

In [4]:
classes = 4
model.fc = torch.nn.Linear(512, classes).to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
loss_fn = torch.nn.CrossEntropyLoss() # multi-class classification model loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

In [6]:
class MyDataset(Dataset):
    def __init__(self,
                 csv_file,      # images could be provided with in a series of directories
                 root_dir,     # images could be provided as a list as well
                 transform = None):  # provide transformation to apply to each image
      """
      Organize the images and the associated labels into two lists.  Potentially create additional
      lists if more complicated information is need.  Important note: images are NOT
      read and stored in this initializer.  They are read in __getitem__ as needed.
      """
      self.csv_file = csv_file # path of csv file
      self.root_dir = root_dir # directory the photos are in
      self.images = pd.read_csv(self.csv_file)
      # Record the transform that may need to be applied.
      self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        '''
        Return a tuple with the data, ground truth label, and any other data
        associated with a single image.
        '''
        img_name = self.images.iloc[idx, 0] # name of image in 1st column
        img_path = os.path.join(self.root_dir, img_name)
        im = Image.open(img_path)

        if self.transform is not None:
            im = self.transform(im)

        """
        label encodes season
        season = {
            0: 'spring'
            1: 'summer'
            2: 'fall'
            3: 'winter'  
        }
        """
        label = self.images.iloc[idx, 1]

        return im, label

In [7]:
os.getcwd()

'/mnt/c/Users/cardon/Desktop/SEM8/RCOS/fashion-forecast'

In [8]:
image_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224)), 
                                        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# transforms.Grayscale(num_output_channels=1)

dataset = MyDataset(csv_file='./filtered_style_stats.csv',
                    root_dir='./yolov5/yolov5/crop-images',
                    transform=image_transforms)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.7, 0.15, 0.15], generator=torch.Generator())

In [9]:
# Referenced from 
# https://towardsdatascience.com/pytorch-basics-sampling-samplers-2a0f29f0bf2a

def get_class_distribution(dataset_obj):
    count_dict = {          \
        'spring': 0,        \
        'summer': 0,        \
        'fall':   0,        \
        'winter': 0         \
    } # type: ignore
    idx_to_class = {        \
        0: 'spring',        \
        1: 'summer',        \
        2: 'fall',          \
        3: 'winter'         \
    }
    
    for idx in range(len(dataset_obj)):
        element = dataset_obj[idx]
        y_lbl = idx_to_class[element[1]]
        count_dict[y_lbl] += 1
    return count_dict

In [10]:
#### CHECKING ITS DECENT ####
total_data = len(train_dataset) + len(val_dataset) + len(test_dataset)
print(f"Length of total data: {total_data}")
print(f"Length of train_dataset: {len(train_dataset)}; {(len(train_dataset)/total_data)*100:.2f}%")
print(f"Length of val_dataset: {len(val_dataset)}; {(len(val_dataset)/total_data)*100:.2f}%")
print(f"Length of test_dataset: {len(test_dataset)}; {(len(test_dataset)/total_data)*100:.2f}%\n")
#print(f"Class Distribution of train_dataset: {get_class_distribution(train_dataset)}")
#print(f"Class Distribution of val_dataset: {get_class_distribution(val_dataset)}")
#print(f"Class Distribution of test_dataset: {get_class_distribution(test_dataset)}")

Length of total data: 267964
Length of train_dataset: 187575; 70.00%
Length of val_dataset: 40195; 15.00%
Length of test_dataset: 40194; 15.00%



In [11]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(train_dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 800 == 0: # every 20000 images run
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

In [13]:
def test(dataloader, model, loss_fn, incorrect_examples, correct_examples):
    size = len(val_dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad(): 
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()

            val = pred.argmax(1).to(device) 
            correct += ((val == y).type(torch.float).sum().item()) 

            if torch.all(torch.eq(val, y)) and len(correct_examples) < 6:
                correct_examples.append(X.cpu())
            if (not torch.all(torch.eq(val, y))) and len(incorrect_examples) < 6:
                incorrect_examples.append(X.cpu())

    val_loss /= num_batches
    correct /= size
    print(f"Val Error ---\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")
    return correct_examples, incorrect_examples

In [14]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    incorrect_examples = []
    correct_examples = []
    correct_examples, incorrect_examples = test(val_dataloader, model, loss_fn, incorrect_examples, correct_examples)
print("Done!\n")

Epoch 1
-------------------------------
loss: 1.420578  [    0/187575]
loss: 1.382192  [25600/187575]
loss: 1.259995  [51200/187575]
loss: 1.219778  [76800/187575]
loss: 1.327443  [102400/187575]
loss: 1.236579  [128000/187575]
loss: 1.517246  [153600/187575]
loss: 1.329134  [179200/187575]
Val Error ---
 Accuracy: 41.0%, Avg loss: 1.252456 

Epoch 2
-------------------------------
loss: 1.158244  [    0/187575]
loss: 1.375487  [25600/187575]
loss: 1.246830  [51200/187575]
loss: 1.216655  [76800/187575]
loss: 1.329320  [102400/187575]
loss: 1.233394  [128000/187575]
loss: 1.516169  [153600/187575]
loss: 1.337231  [179200/187575]
Val Error ---
 Accuracy: 41.2%, Avg loss: 1.252164 

Epoch 3
-------------------------------
loss: 1.157117  [    0/187575]
loss: 1.373440  [25600/187575]
loss: 1.241705  [51200/187575]
loss: 1.218745  [76800/187575]
loss: 1.331375  [102400/187575]
loss: 1.234507  [128000/187575]
loss: 1.511586  [153600/187575]
loss: 1.338363  [179200/187575]
Val Error ---
 Acc

In [15]:
correct_examples, incorrect_examples = test(test_dataloader, model, loss_fn, incorrect_examples, correct_examples)